In [38]:
import pandas as pd 
import boto3
import json
from io import StringIO
from io import BytesIO
import matplotlib.pyplot as plt

### 0. Dataframe Ingestion and prep
#### Ingest now for later usage

In [48]:
###Initialize a session using Amazon S3
s3 = boto3.client('s3')

###Define the S3 bucket
bucket_name = 'rearc-data-quest-conor'

###Define the S3 bucket
p1_file_name = 'bls_data/pr.data.0.Current'

###Get the file from S3 and read it into a data farme
try:
    file_obj = BytesIO()
    s3.download_fileobj(bucket_name, p1_file_name, file_obj)
    file_obj.seek(0)
    p1_df = pd.read_csv(file_obj, sep='\s+')
except Exception as e:
    print(e)

###Define file name for part 2 processing
p2_file_name = 'usa_data/06_24_2024/usa_data_api.json'

###Get the file object from S3
p2_file_obj = s3.get_object(Bucket=bucket_name, Key=p2_file_name)
p2_file_content = p2_file_obj['Body'].read().decode('utf-8')

###Load JSON data using pandas
p2_json_content = json.loads(p2_file_content)
p2_json_content = p2_json_content['data']
part_two_df = pd.json_normalize(p2_json_content)
print(part_two_df)
print(p1_df)

  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2022  2022   331097593  united-states
1   01000US  United States     2021  2021   329725481  united-states
2   01000US  United States     2020  2020   326569308  united-states
3   01000US  United States     2019  2019   324697795  united-states
4   01000US  United States     2018  2018   322903030  united-states
5   01000US  United States     2017  2017   321004407  united-states
6   01000US  United States     2016  2016   318558162  united-states
7   01000US  United States     2015  2015   316515021  united-states
8   01000US  United States     2014  2014   314107084  united-states
9   01000US  United States     2013  2013   311536594  united-states
         series_id  year period    value footnote_codes
0      PRS30006011  1995    Q01    2.600            NaN
1      PRS30006011  1995    Q02    2.100            NaN
2      PRS30006011  1995    Q03    0.900            NaN
3      PRS3000601

### 1. Basic Statistics
#### Find the mean and standard deviation of the US population between 2013-2018 using Part 2 Data
#### Must use pandas

In [14]:
###Filtering data for 2013-2018
part_two_df['Year'] = pd.to_datetime(part_two_df['Year']).dt.year
part_two_filtered_df = part_two_df[(part_two_df['Year'].between(2013,2018))]
needed_columns = ['Population']
###filtering for just the needed columns
new_df = part_two_filtered_df[needed_columns]
###Getting actual stats
mean_std = new_df.agg(['mean','std']).reset_index()
print(mean_std)

  ID Nation         Nation  ID Year  Year  Population    Slug Nation
4   01000US  United States     2018  2018   322903030  united-states
5   01000US  United States     2017  2017   321004407  united-states
6   01000US  United States     2016  2016   318558162  united-states
7   01000US  United States     2015  2015   316515021  united-states
8   01000US  United States     2014  2014   314107084  united-states
9   01000US  United States     2013  2013   311536594  united-states
  index    Population
0  mean  3.174374e+08
1   std  4.257090e+06


### Mean population of the US between 2013-2018 = 3.174374e+08 or 317,437,400.
### Standard Deviation of the US poulation between 2013-2018 = 4.257090e+06 or 4,257,090.

### 2. Part 1 Report
#### Create report that will show each series id and their best year by value.

In [44]:
###Summing DF beforehand
df_summed = p1_df.groupby(['series_id','year'])['value'].sum().reset_index()
###Finding max
idx = df_summed.groupby('series_id')['value'].idxmax()
###Matching
report = df_summed.loc[idx, ['series_id', 'year', 'value']].reset_index()
print(report)

     index    series_id  year    value
0       27  PRS30006011  2022   20.500
1       57  PRS30006012  2022   17.100
2       63  PRS30006013  1998  704.125
3      105  PRS30006021  2010   17.600
4      135  PRS30006022  2010   12.500
..     ...          ...   ...      ...
277   8182  PRS88003192  2002  282.800
278   8233  PRS88003193  2023  767.007
279   8262  PRS88003201  2022   36.100
280   8292  PRS88003202  2022   28.900
281   8323  PRS88003203  2023  582.391

[282 rows x 4 columns]


### 3. Combining both dataframe
#### Create a report contating, series_id, year, period, value, and yearly population, of Series PRS30006032 and Period Q01


In [54]:
###Filtering p1 df for just what we need.
filtered_p1 = p1_df[(p1_df['series_id'] == 'PRS30006032') & (p1_df['period'] == 'Q01')]
filtered_p1['year'] = pd.to_datetime(filtered_p1['year'], format='%Y')
part_two_df = part_two_df.rename(columns={'Year':'year'})
part_two_df['year'] = pd.to_datetime(part_two_df['year'], format='%Y')
report_2 = filtered_p1.merge(part_two_df, on='year', how='left')
print(report_2[['series_id', 'year', 'period', 'value', 'Population']])

      series_id       year period  value   Population
0   PRS30006032 1995-01-01    Q01    0.0          NaN
1   PRS30006032 1996-01-01    Q01   -4.4          NaN
2   PRS30006032 1997-01-01    Q01    2.7          NaN
3   PRS30006032 1998-01-01    Q01    1.0          NaN
4   PRS30006032 1999-01-01    Q01   -4.1          NaN
5   PRS30006032 2000-01-01    Q01    0.5          NaN
6   PRS30006032 2001-01-01    Q01   -6.5          NaN
7   PRS30006032 2002-01-01    Q01   -6.7          NaN
8   PRS30006032 2003-01-01    Q01   -5.6          NaN
9   PRS30006032 2004-01-01    Q01    2.1          NaN
10  PRS30006032 2005-01-01    Q01   -0.6          NaN
11  PRS30006032 2006-01-01    Q01    1.8          NaN
12  PRS30006032 2007-01-01    Q01   -0.7          NaN
13  PRS30006032 2008-01-01    Q01   -3.4          NaN
14  PRS30006032 2009-01-01    Q01  -21.0          NaN
15  PRS30006032 2010-01-01    Q01    3.4          NaN
16  PRS30006032 2011-01-01    Q01    1.7          NaN
17  PRS30006032 2012-01-01  

C:\Users\cpow2\AppData\Local\Temp\ipykernel_6572\1917195880.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_p1['year'] = pd.to_datetime(filtered_p1['year'], format='%Y')
